<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Tutorial

*Deze tutorial is een aangepaste versie van de officiële [Cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html) en de [Cython in Jupyter Notebooks instructies](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#using-the-jupyter-notebook). Het compileren van Cython files voor gebruik in reguliere python projecten is in dit notebook achterwege gelaten. Dit zal tijdens de kennissessie aan bod komen.*

## The Basics of Cython

The fundamental nature of Cython can be summed up as follows: Cython is Python with C data types.

Cython is Python: Almost any piece of Python code is also valid Cython code. (There are a few Limitations, but this approximation will serve for now.) The Cython compiler will convert it into C code which makes equivalent calls to the Python/C API.

But Cython is much more than that, because parameters and variables can be declared to have C data types. Code which manipulates Python values and C values can be freely intermixed, with conversions occurring automatically wherever possible. Reference count maintenance and error checking of Python operations is also automatic, and the full power of Python’s exception handling facilities, including the try-except and try-finally statements, is available to you – even in the midst of manipulating C data.

## Cython and Jupyter Notebooks

We will use the following python code for our first example. 

In [ ]:
a = 0
for i in range(10):
  a += i
print(a)

Cython can be used conveniently and interactively from a web browser through the Jupyter notebook. To enable support for Cython compilation load the Cython extension from within the Jupyter notebook:

In [ ]:
%load_ext Cython

Then, prefix a cell with the %%cython marker to compile it with Cython. We can then use C data types in our code.

In [ ]:
%%cython

cdef int a = 0
for i in range(10):
  a += i
print(a)

This is not very exciting yet, so lets look at another example.

## Fibonacci Fun


From the official Python tutorial a simple fibonacci function is defined as:

In [ ]:
def fib(n):
  """Return the Fibonacci series up to n."""
  a, b = 0, 1
  r = []
  while b < n:
    r.append(b)
    a, b = b, a + b

  return r

In [ ]:
fib(2000)

We can compile the function in Cython without any changes.

In [ ]:
%%cython

def c_fib(n):
  """Return the Fibonacci series up to n."""
  a, b = 0, 1
  r = []
  while b < n:
    r.append(b)
    a, b = b, a + b

  return r

Lets time both functions to see the difference.

In [ ]:
%timeit x = fib(2000)

In [ ]:
%timeit x = c_fib(2000)

It already is a lot faster. The next section will cover more advanced Cython features, which can be used to make it run even faster.

## Primes

Here’s a small example showing some of what can be done. It’s a routine for finding prime numbers. You tell it how many primes you want, and it returns them as a Python list.

In [ ]:
%%cython
def primes(int nb_primes):
  cdef int n, i, len_p
  cdef int p[1000]
  if nb_primes > 1000:
    nb_primes = 1000

  len_p = 0  # The current number of elements in p.
  n = 2
  while len_p < nb_primes:
    # Is n prime?
    for i in p[:len_p]:
      if n % i == 0:
        break

    # If no break occurred in the loop, we have a prime.
    else:
      p[len_p] = n
      len_p += 1
    n += 1

  # Let's return the result in a python list:
  result_as_list  = [prime for prime in p[:len_p]]
  return result_as_list

You’ll see that it starts out just like a normal Python function definition, except that the parameter `nb_primes` is declared to be of type `int`. This means that the object passed will be converted to a C integer (or a `TypeError` will be raised if it can’t be).

Now, let’s dig into the core of the function:

```
cdef int n, i, len_p
cdef int p[1000]
```

Lines 2 and 3 use the `cdef` statement to define some local C variables. The result is stored in the C array p during processing, and will be copied into a Python list at the end (line 22).

**Note** *You cannot create very large arrays in this manner, because they are allocated on the C function call stack, which is a rather precious and scarce resource. To request larger arrays, or even arrays with a length only known at runtime, you can learn how to make efficient use of C memory allocation, Python arrays or NumPy arrays with Cython.*

```
if nb_primes > 1000:
  nb_primes = 1000
```

As in C, declaring a static array requires knowing the size at compile time. We make sure the user doesn’t set a value above 1000 (or we would have a segmentation fault, just like in C).

```
len_p = 0  # The number of elements in p
n = 2
while len_p < nb_primes:
```

Lines 7-9 set up for a loop which will test candidate numbers for primeness until the required number of primes has been found.

```
# Is n prime?
for i in p[:len_p]:
  if n % i == 0:
    break
```

Lines 11-12, which try dividing a candidate by all the primes found so far, are of particular interest. Because no Python objects are referred to, the loop is translated entirely into C code, and thus runs very fast. You will notice the way we iterate over the `p` C array.

```
for i in p[:len_p]:
```

The loop gets translated into a fast C loop and works just like iterating over a Python list or NumPy array. If you don’t slice the C array with `[:len_p]`, then Cython will loop over the 1000 elements of the array.

```
# If no break occurred in the loop
else:
  p[len_p] = n
  len_p += 1
n += 1
```

If no breaks occurred, it means that we found a prime, and the block of code after the `else` line 16 will be executed. We add the prime found to `p`. If you find having an `else` after a for-loop strange, just know that it’s a lesser known features of the Python language, and that Cython executes it at C speed for you. If the for-else syntax confuses you, see this excellent [blog post](https://s16h.medium.com/the-forgotten-optional-else-in-python-loops-90d9c465c830).

```
# Let's put the result in a python list:
result_as_list  = [prime for prime in p[:len_p]]
return result_as_list
```

In line 22, before returning the result, we need to copy our C array into a Python list, because Python can’t read C arrays. Cython can automatically convert many C types from and to Python types, as described in the [documentation on type conversion](https://cython.readthedocs.io/en/latest/src/userguide/language_basics.html#type-conversion), so we can use a simple list comprehension here to copy the C `int` values into a Python list of Python `int` objects, which Cython creates automatically along the way. You could also have iterated manually over the C array and used `result_as_list.append(prime)`, the result would have been the same.

You’ll notice we declare a Python list exactly the same way it would be in Python. Because the variable `result_as_list` hasn’t been explicitly declared with a type, it is assumed to hold a Python object, and from the assignment, Cython also knows that the exact type is a Python list.

Finally, at line 23, a normal Python return statement returns the result list.

Now try calling the function in a normal cell:



In [ ]:
print(primes(10))

See, it works! And if you’re curious about how much work Cython has saved you, take a look at the C code generated for this module.

Cython has a way to visualise where interaction with Python objects and Python’s C-API is taking place. For this, we can add `-- annotate` to the `%%cython` line.

In [ ]:
%%cython --annotate
def primes(int nb_primes):
  cdef int n, i, len_p
  cdef int p[1000]
  if nb_primes > 1000:
    nb_primes = 1000

  len_p = 0  # The current number of elements in p.
  n = 2
  while len_p < nb_primes:
    # Is n prime?
    for i in p[:len_p]:
      if n % i == 0:
          break

    # If no break occurred in the loop, we have a prime.
    else:
      p[len_p] = n
      len_p += 1
    n += 1

  # Let's return the result in a python list:
  result_as_list  = [prime for prime in p[:len_p]]
  return result_as_list

If a line is white, it means that the code generated doesn’t interact with Python, so will run as fast as normal C code. The darker the yellow, the more Python interaction there is in that line. Those yellow lines will usually operate on Python objects, raise exceptions, or do other kinds of higher-level operations than what can easily be translated into simple and fast C code. The function declaration and return use the Python interpreter so it makes sense for those lines to be yellow. Same for the list comprehension because it involves the creation of a Python object. But the line `if n % i == 0:`, why? We can examine the generated C code to understand. Click on the *+* on line 12 to expand it.

We can see that some checks happen. Because Cython defaults to the Python behavior, the language will perform division checks at runtime, just like Python does. You can deactivate those checks by using the [compiler directives](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives).

Now let’s see if, even if we have division checks, we obtained a boost in speed. Let’s write the same program, but Python-style:

In [ ]:
def primes_python(nb_primes):
  p = []
  n = 2
  while len(p) < nb_primes:
    # Is n prime?
    for i in p:
      if n % i == 0:
          break

    # If no break occurred in the loop
    else:
      p.append(n)
    n += 1
  return p

We can also compile this function as is with cython:

In [ ]:
%%cython
def primes_python_compiled(nb_primes):
  p = []
  n = 2
  while len(p) < nb_primes:
    # Is n prime?
    for i in p:
      if n % i == 0:
        break

    # If no break occurred in the loop
    else:
      p.append(n)
    n += 1
  return p

We time all functions.

In [ ]:
%timeit primes_python(1000)

In [ ]:
%timeit primes_python_compiled(1000)

In [ ]:
%timeit primes(1000)

The cythonize version of `primes_python` is more then 2 times faster than the Python one, without changing a single line of code. The Cython version is almost 20 times faster than the Python version! What could explain this?

Multiple things:


*   In this program, very little computation happen at each line. So the overhead of the python interpreter is very important. It would be very different if you were to do a lot computation at each line. Using NumPy for example.
*   Data locality. It’s likely that a lot more can fit in CPU cache when using C than when using Python. Because everything in python is an object, and every object is implemented as a dictionary, this is not very cache friendly.

Usually the speedups are between 2x to 1000x. It depends on how much you call the Python interpreter. As always, remember to profile before adding types everywhere. Adding types makes your code less readable, so use them with moderation.

## Try it yourself

Try to add typing to the fibbonaci function.

In [ ]:
%%cython
def c_fib_typed(n):
  """Return the Fibonacci series up to n."""
  a, b = 0, 1
  r = []
  while b < n:
    r.append(b)
    a, b = b, a + b

  return r

In [ ]:
%timeit x = c_fib_typed(2000)

## Further reading and useful links

Doorlezen: [Cython Function declarations](https://notes-on-cython.readthedocs.io/en/latest/function_declarations.html)

Ter referentie: [Full Cython documentation](https://cython.readthedocs.io/en/latest/index.html)